In [1]:
import numpy as np 
from scipy.integrate import odeint
import os, sys, warnings
from pathlib import Path
from os.path import dirname, realpath
script_dir = Path(dirname(realpath('.')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
import utility as ut
import surrogate_nn as srnn
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
warnings.filterwarnings('ignore')

In [ ]:
DTYPE = 'float64'
L63_data_path = '../data/L63-trajectories'
save_folder='../data/adaptive-rate'
log_interval = 100
milestones = [10*2**n for n in range(15)]
learning_rate = 1e-3
drop = 0.7
steps = int(2e5)
save_interval = 100
N = 20000
L0 = 0.4
L1 = 3.5
beta = 4e-5
min_change = -(1.+0.1)**(1./log_interval) + 1.
train = np.load(f'{L63_data_path}/train.npy').astype(DTYPE)
test = np.load(f'{L63_data_path}/test.npy')[:, :, :1000].astype(DTYPE)

model = srnn.SurrogateModel_NN(3, 100, name='nn', save_folder=save_folder)
model.learn(train[:, :N], steps, 1e-3, batch_size='GD', log_interval=100, save_interval=100,\
           max_delta=0.1, min_change=min_change, constant_rate=False, update_frequency=1000)
tau_f_rmse, tau_f_se, rmse, se = model.compute_tau_f(test[:100], error_threshold=0.05)
tau_f_1 = tau_f_se.mean()
tau_f_1

step: 0 loss: 15322181.589488 time: 0.0118 lr: 0.001000,  change: nan%
step: 100 loss: 7310698.355593 time: 0.8728 lr: 0.001000,  change: -52.29%
step: 200 loss: 3526905.591494 time: 1.7229 lr: 0.001000,  change: -51.76%
step: 300 loss: 2084381.631710 time: 2.5737 lr: 0.001000,  change: -40.90%
step: 400 loss: 1475123.763965 time: 3.4254 lr: 0.001000,  change: -29.23%
step: 500 loss: 1114289.204386 time: 4.2787 lr: 0.001000,  change: -24.46%
step: 600 loss: 829287.285342 time: 5.1424 lr: 0.001000,  change: -25.58%
step: 700 loss: 611054.717872 time: 6.0402 lr: 0.001000,  change: -26.32%
step: 800 loss: 443415.554516 time: 6.9347 lr: 0.001000,  change: -27.43%
step: 900 loss: 314184.021551 time: 7.8263 lr: 0.001000,  change: -29.14%
step: 1000 loss: 222868.429417 time: 8.7030 lr: 0.001000,  change: -29.06%
step: 1100 loss: 163229.465577 time: 9.5720 lr: 0.001000,  change: -26.76%
step: 1200 loss: 124076.933189 time: 10.4299 lr: 0.001000,  change: -23.99%
step: 1300 loss: 97654.238198 ti

In [ ]:
-0.0006 > min_change

In [ ]:
a = torch.rand(2, 2)
a

In [ ]:
torch.sum(a**2)

In [ ]:
torch.norm(a)**2

In [ ]:
0.1795, 1.105